### Generate stimulus:

In [48]:
import numpy as np
from scipy.signal import convolve2d

actBits = 3
weightBits = 3

nActsX = 10
nActsY = 10
nWeights = 3

a = np.random.randint(-2**(actBits-1),2**(actBits-1)-1,size=(nActsX,nActsY))
w = np.random.randint(-2**(weightBits-1),2**(weightBits-1)-1,size=(nWeights,nWeights))
o = convolve2d(a,w[::-1].T[::-1].T)

In [49]:
a

array([[ 1, -2,  1, -2, -1, -4,  1, -4,  2,  1],
       [ 0, -4,  1, -4, -1,  0,  0,  2,  1, -2],
       [ 0,  1, -4,  0,  1,  0,  0, -1,  2,  1],
       [-4, -1, -1, -2, -1, -1,  0, -3, -1, -4],
       [-3, -2, -3,  2, -3,  0, -3, -3, -3,  1],
       [-4, -1, -1, -1,  1,  2, -2,  1, -1,  1],
       [ 0,  1, -4, -3, -1,  2,  1, -1, -4, -3],
       [-3, -1,  2, -4,  0,  1,  1,  2,  1,  2],
       [ 0,  0, -1,  0,  1, -1, -3, -3, -2, -4],
       [ 1, -3,  0, -4, -3, -4, -3, -4, -2,  2]])

In [50]:
w

array([[ 0, -4,  1],
       [ 1,  1, -3],
       [-4, -2, -2]])

In [51]:
o

array([[ -2,   2,  -2,  10,   2,  18,  10,  22,   0,  10, -10,  -4],
       [ -3,  15,   2,  27,   8,  27,  -4,   5, -15, -11,   3,   9],
       [  1,   4,   8,   7,  21,  -7,  12, -12,  15,   0, -15,  -6],
       [  8,   3,  50,  -1,  18,  17,   7,  15,  -6,  14,  23,  17],
       [ 18,  10,  12,  30,  15,  -6,  16,  19,  30,  17,   1,  -8],
       [ 13,  28,  27,  -1,  19,   0,   6,  -3,  22, -13,  16,  -3],
       [  9,   7,   9,  25,   8,  16,   4,  -2,  17,  28,  18,  13],
       [  2,  20,  26,  17,   1,   2, -16,   5,  -3,  -5, -19, -11],
       [  9,   1, -16,  28,  11,  -1,  -5,   7,  22,  34,  31,  18],
       [ -5,  15,  11,   7,  26,  35,  32,  33,  17,  21, -10, -12],
       [ -3,  10,  -3,  13,   6,   0,   3,  14,   9,  -8,  16,   2],
       [  1,  -7,  12,  -4,  13,   8,  13,   8,  14,  10,  -8,   0]])

### Prepare scan chain for multicast ID

In [61]:
ids = w
for x,idx in enumerate(ids):
    for y,id in enumerate(idx):
        ids[x][y] = x + y
ids.flatten()

array([0., 1., 2., 1., 2., 3., 2., 3., 4.])

### Save stimulus:

In [63]:
path = '/home/lquizon/lawrence-workspace/eyeriss/tb/cluster'
np.savetxt(path + '/a.txt',a,'%i')
np.savetxt(path + '/w.txt',w,'%i')
np.savetxt(path + '/o.txt',o,'%i')
np.savetxt(path + '/ids.txt',ids,'%i')

### Have a look at existing stimulus:

In [55]:
import numpy as np
path = '.'
a = np.loadtxt(path + '/a.txt')
w = np.loadtxt(path + '/w.txt')
o= np.loadtxt(path+'/o.txt')  